### To use this notebook, follow my process below. This focuses on doing this locally on your own machine. API will be covered later

## 0.0 Import the required packages

In [1]:
import altair as alt
import pandas as pd


## 1.0. Preparing the Data for the base of the map:
   - Load the hexjson file containing the data you want to work with.
   - Make sure the hexjson file is in the correct format and accessible from the notebook.
   - You can use the `json` module to load the hexjson file into a Python dictionary.

### 1.1 Converting the JSON to a DataFrame:
   - Once you have loaded the hexjson data into a dictionary, you can convert it to a DataFrame using the `pd.DataFrame()` function from the pandas library.
   - Pass the dictionary as the argument to the `pd.DataFrame()` function to create the DataFrame.
   - Assign the resulting DataFrame to a variable for further processing.


In [2]:
alt_data = pd.read_json('/home/asia/code/willgreen93/UK_election/interface/data/uk-constituencies-2019-BBC.hexjson')
alt_df = pd.DataFrame(alt_data)


### 1.2 Cleaning up the columns
In this case, the hex file we're using is from OpenInnovations https://open-innovations.org/projects/hexmaps/constituencies/ -- check what and where to get the latitutde and longitude, for this file it will be 'r' and 'q' instead of the usual lat and lon values. You will also be able to use the 'region' key to merge your data with the hex map provided

In [3]:
## you can check values with ###
# alt_df['hexes'][0] #this is specific to this json file

alt_df[['n', 'r', 'q', 'region']] = \
    alt_df['hexes'].apply(lambda x: pd.Series([x.get('n'), x.get('r'), x.get('q'), x.get('region')]))

# #reset index to prepare for joining to results
new_df = alt_df.rename_axis('constituency_id').reset_index().drop(columns=['hexes'])


## 2.0 Merging the DataFrame with the Prediction (Voting Results):

If you have a separate prediction or voting results dataset that you want to merge with the main DataFrame, follow these steps:
- Load the prediction dataset into a separate DataFrame using the same process as described in step 2.
- Ensure that both DataFrames have a common column that can be used as a key for merging, you can use the constituency ID for this case
- Use the `pd.merge()` function to merge the two DataFrames based on the common column.
- Assign the merged DataFrame to a new variable for further analysis.


In [5]:
#adding scotland and NI results, as they remain static and new info is not available
extra_cols = pd.read_csv("https://storage.googleapis.com/uk_election_model/Scotland_NI_results.csv")
extra_cols = pd.DataFrame(extra_cols)


In [7]:
extra_cols.rename(columns={'ons_id': 'constituency_id'}, inplace=True)


In [12]:
extra_cols.keys()
extra_cols_clean = extra_cols.drop(columns=['snp', 'dup', 'sf', 'sdlp', 'uup', 'alliance'])


In [17]:
api_url = "https://ukelection-image-ne4yelgixa-no.a.run.app/predict"
preds = pd.read_json(api_url)
pred_df = pd.DataFrame(preds)
pred_df


,constituency_id,con_votes,lab_votes,lib_votes,oth_votes,winning_party
0,E14000530,20093,18101,5770,4462,con
1,E14000531,23505,12611,3109,6019,con
2,E14000532,16040,23602,4183,5170,lab
3,E14000533,19529,16892,2823,5435,con
4,E14000534,16530,11822,12229,5165,con
...,...,...,...,...,...,...
568,W07000076,5915,22010,693,15027,lab
569,W07000077,6924,20804,-836,13944,lab
570,W07000078,14651,25283,1002,3280,lab
571,W07000079,6118,28431,885,10080,lab


In [23]:
combined_df = pd.concat([extra_cols_clean, pred_df], ignore_index=True)


In [27]:
combined_df['winning_party'] = combined_df['winning_party'].str.lower()


In [29]:
combined_df.winning_party.unique()


array(['snp', 'con', 'dup', 'sf', 'sdlp', 'lib', 'lab', 'alliance', 'oth'],
      dtype=object)

## 3.0 Edit params.py to fill out with the URL and the params necessary for the streamlit app

In [23]:
df.incumbent_party.unique( )


array(['labour', 'conservative', 'other_parties', 'liberal_democrats'],
      dtype=object)

In [24]:
conser = (df['incumbent_party'] == 'conservative').sum()
labour = (df['incumbent_party'] == 'labour').sum()
other_part = (df['incumbent_party'] == 'other_parties').sum()
lib_dem = (df['incumbent_party'] == 'liberal_democrats').sum()


In [25]:
conser, labour, other_part, lib_dem


(316, 260, 62, 12)

In [ ]:
st.altair_chart(
    alt.Chart().mark_text(
    align="left",
    baseline="bottom",
    fontSize=14,
    fontWeight=600,
    color='coral'
).encode(
    x=alt.value(410),  # pixels from left
    y=alt.value(290),  # pixels from top
    text=alt.value(["Summary:", f'{conser, labour, other_part, lib_dem}'])
)
)


In [26]:
df.head(10)


,constituency_id,constituency,country,incumbent_party,layout,n,r,q,region
0,W07000049,ABERAVON,Wales,labour,odd-q,Aberavon,11,8,W92000004
1,W07000058,ABERCONWY,Wales,conservative,odd-q,Aberconwy,18,9,W92000004
2,S14000001,ABERDEEN NORTH,Scotland,other_parties,odd-q,Aberdeen North,40,15,S92000003
3,S14000002,ABERDEEN SOUTH,Scotland,conservative,odd-q,Aberdeen South,40,16,S92000003
4,S14000003,AIRDRIE AND SHOTTS,Scotland,other_parties,odd-q,Airdrie and Shotts,36,13,S92000003
5,E14000530,ALDERSHOT,England,conservative,odd-q,Aldershot,7,16,E12000008
6,E14000531,ALDRIDGE-BROWNHILLS,England,conservative,odd-q,Aldridge-Brownhills,20,17,E12000005
7,E14000532,ALTRINCHAM AND SALE WEST,England,conservative,odd-q,Altrincham and Sale West,18,13,E12000002
8,W07000043,ALYN AND DEESIDE,Wales,labour,odd-q,Alyn and Deeside,16,11,W92000004
9,E14000533,AMBER VALLEY,England,conservative,odd-q,Amber Valley,20,21,E12000004


In [ ]:

source = pd.DataFrame({
    "a": ["A", "B", "C"],
    "b": [28, 55, 43]
})

bar = alt.Chart(df).mark_bar().encode(
    y="a:N",
    x=alt.X("b:Q").scale(domain=[0, 60])
)
text = bar.mark_text(
    align="left",
    baseline="middle",
    dx=3
).encode(text="b")

bar + text


In [28]:
map = (
    alt.Chart(df)
    .mark_square()
    .encode(
        x=alt.X("q").scale(zero=False).axis(None),
        y=alt.Y("r").scale(zero=False).axis(None),
        color=colours_obj.legend(
            title="Incumbent Party",
        ),
        size=alt.value(65),
        tooltip=["n:N"],
    )
    .properties(width=700, height=650)
    .configure_axis(grid=False)
    .configure_view(strokeWidth=0)
)

bar_ch = (
    alt.Chart(df)
    .mark_bar()
    .encode(
        x=alt.X("incumbent_party:N", title="Incumbent Party"),
        y=alt.Y("count():Q", title="Count"),
        tooltip=[alt.Tooltip("count()", title="Count")],
        color="incumbent_party:N",
    )
    .properties(
        title="Total Count of Each Incumbent Party",
        width=400,  # Adjust the width as needed
    )
)

st.altair_chart(map)
st.altair_chart(bar_ch)


NameError: name 'colours_obj' is not defined

In [ ]:
alt.Chart(source).mark_bar().encode(
    alt.X("sum(people):Q").title("Population"),
    alt.Y("age:O"),
).transform_filter(
    datum.year == 2000
).properties(height=alt.Step(20))


In [ ]:
###WORKING APP
map = (
    alt.Chart(df)
    .mark_square()
    .encode(
        x=alt.X("q").scale(zero=False).axis(None),
        y=alt.Y("r").scale(zero=False).axis(None),
        color=colours_obj.legend(
            title="Incumbent Party",
        ),
        size=alt.value(65),
        tooltip=["n:N"],
    )
    .properties(width=700, height=650)
    .configure_axis(grid=False)
    .configure_view(strokeWidth=0)
)

bar_ch = (
    alt.Chart(df)
    .mark_bar()
    .encode(
        x=alt.X("count():Q", title="Count"),
        y=alt.Y("incumbent_party:N", title="Incumbent Party"),
        tooltip=[alt.Tooltip("count()", title="Count")],
        color="incumbent_party:N",
    )
    .properties(
        title="Total Count of Each Incumbent Party",
        width=400,  # Adjust the width as needed
    )
)

st.altair_chart(map)
st.altair_chart(bar_ch)


In [ ]:
brush = alt.selection(type="interval")

map = (
    alt.Chart(df)
    .mark_square()
    .encode(
        x=alt.X("q").scale(zero=False).axis(None),
        y=alt.Y("r").scale(zero=False).axis(None),
        color=colours_obj.legend(title="Incumbent Party", orient="bottom").condition(brush, "incumbent_party:N", alt.value("lightgray")
        ),
        size=alt.value(65),
        tooltip=["n:N"],
    )
    .properties(width=500, height=650)
    .configure_axis(grid=False)
    .configure_view(strokeWidth=0)
)


bar_ch = (
    alt.Chart(df)
    .mark_bar()
    .encode(
        x=alt.X("count():Q", title="Total Count"),
        y=alt.Y("incumbent_party:N", title="Incumbent Party", axis=None),
        tooltip=[alt.Tooltip("count()")],
        color=colours_obj.legend(None),
    )
    .properties(
        title="Total Count of Each Incumbent Party",
        width=400,
    )
)


In [ ]:
#conditional sliders

conservative_rating = st.sidebar.slider(
    "Conservative Rating", min_value=0, max_value=100, value=0
)
labor_party_rating = st.sidebar.slider(
    "Labor Party Rating", min_value=0, max_value=100 - conservative_rating, value=0
)
libdem_party_rating = st.sidebar.slider(
    "Lib Dem Rating",
    min_value=0,
    max_value=100 - (labor_party_rating + conservative_rating),
    value=0,
)
other_party_rating = st.sidebar.slider(
    "Other Parties",
    min_value=0,
    max_value=100 - (labor_party_rating + conservative_rating + libdem_party_rating),
    value=0,
)
let_chaos_reign = st.sidebar.slider(
    "Let Chaos Reign", min_value=0, max_value=100, value=50
)


In [29]:
import requests

def fetch_data_from_api(api_url, params=None, headers=None):
    response = requests.get(api_url, params=params, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        return None


api_url = "https://ukelection-image-ne4yelgixa-no.a.run.app/predict"

# Fetch data from the API
api_data = fetch_data_from_api(api_url)


In [31]:
data_source = pd.DataFrame(api_data)


In [35]:
data_source.winning_party.unique()


array(['con', 'lab', 'oth', 'lib'], dtype=object)

In [34]:
data_source


,constituency_id,con_votes,lab_votes,lib_votes,oth_votes,winning_party
0,E14000530,20096,18101,5770,4462,con
1,E14000531,23505,12611,3109,6019,con
2,E14000532,16239,23602,4183,5170,lab
3,E14000533,19529,16850,3002,5599,con
4,E14000534,16530,11981,12229,5025,con
...,...,...,...,...,...,...
568,W07000076,6164,22658,786,12173,lab
569,W07000077,8483,21567,-719,13944,lab
570,W07000078,14440,25108,1283,3280,lab
571,W07000079,5978,28270,818,10013,lab


In [37]:
pd.merge(data_source, df, on='constituency_id', how='left')


,constituency_id,con_votes,lab_votes,lib_votes,oth_votes,winning_party,constituency,country,incumbent_party,layout,n,r,q,region
0,E14000530,20096,18101,5770,4462,con,ALDERSHOT,England,conservative,odd-q,Aldershot,7,16,E12000008
1,E14000531,23505,12611,3109,6019,con,ALDRIDGE-BROWNHILLS,England,conservative,odd-q,Aldridge-Brownhills,20,17,E12000005
2,E14000532,16239,23602,4183,5170,lab,ALTRINCHAM AND SALE WEST,England,conservative,odd-q,Altrincham and Sale West,18,13,E12000002
3,E14000533,19529,16850,3002,5599,con,AMBER VALLEY,England,conservative,odd-q,Amber Valley,20,21,E12000004
4,E14000534,16530,11981,12229,5025,con,ARUNDEL AND SOUTH DOWNS,England,conservative,odd-q,Arundel and South Downs,5,22,E12000008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568,W07000076,6164,22658,786,12173,lab,CAERPHILLY,Wales,labour,odd-q,Caerphilly,10,11,W92000004
569,W07000077,8483,21567,-719,13944,lab,ISLWYN,Wales,labour,odd-q,Islwyn,12,10,W92000004
570,W07000078,14440,25108,1283,3280,lab,VALE OF GLAMORGAN,Wales,conservative,odd-q,Vale of Glamorgan,9,8,W92000004
571,W07000079,5978,28270,818,10013,lab,CARDIFF WEST,Wales,labour,odd-q,Cardiff West,9,9,W92000004
